# 1. Thông tin chung
Thông tin của nhóm

# 2. Thông tin về dữ liệu

## 2.1 Thông tin chung
- Thu thập ở đâu?
- License ra sao?
- Người ta thu thập ra sao?

## 2.2 Khám phá dữ liệu

# 3. Câu hỏi cần trả lời

## 3.1 Câu hỏi 1

### 3.1.1 Nêu vấn đề

### 3.1.2 Data Preprocessing

### 3.1.3 Analysis, Visualization

### 3.1.4 Kết luận

## 3.2 Câu hỏi 2

### 3.2.1 Nêu vấn đề

### 3.2.2 Data Preprocessing

### 3.2.3 Analysis, Visualization

### 3.2.4 Kết luận

## 3.3 Phương hướng phát triển nếu còn thời gian

# 4. Cảm nhận sau khi kết thúc môn học

## 4.1 Lê Đức Huy - 19120080

## 4.2 Mai Duy Nam - 19120298

# 5. Tài liệu tham khảo